In [ ]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel('Questions with labels.xlsx')

# 创建一个空的DataFrame来保存结果
result_df = pd.DataFrame(columns=['Question', 'Question_type', 'Entity1:Object','Entity2:Spectrum'])

# 定义一个列表，包含我们感兴趣的类型
interested_types = ['Feature Processing Method', 'machine learning Method', 'Evaluated metrics']

# 遍历原始DataFrame的每一行
for index, row in df.iterrows():
    if row[2] in interested_types:
        # 如果第三列的值是我们感兴趣的类型之一
        new_row = pd.DataFrame({
            'Question': [row[1]],
            'Question_type': [row[2]],
            'Entity1:Object': [row[3]],
            'Entity2:Spectrum':[row[4]]
        })
        result_df = pd.concat([result_df, new_row], ignore_index=True)

# 保存结果到新的Excel文件
result_df.to_excel('Answers_type2.xlsx', index=False)


In [ ]:
import pandas as pd

# 定义映射关系
conversion_dict = {
    '1': ['ultravio spectroscopy(uv)'],
    '2': ['Visible light spectroscopy(vis)'],
    '3.1': ['near-infrared spectroscopy(nir)'],
    '3.2': ['mid-infrared spectroscopy(mir)'],
    '3.3': ['Fourier transform infrared spectroscopy(ftir)'],
    '3.4': [ 'long-wave infrared spectroscopy(lir)'],
    '4': ['Raman spectroscopy'],
    '5': ['Laser-induced breakdown spectroscopy(LIBS)'],
    '6': ['terahertz spectroscopy(THz)'],
    '7': ['fluorescence spectroscopy'],
    '8': ['hyperspectral imaging technology'],
    '9': ['else']
}

# 读取 Excel 文件
file_path = 'Label_ver7_updated.xlsx'  # 更改为你的文件路径
df = pd.read_excel(file_path)

# 转换数字到相应的文字列表
def convert_numbers_to_text(num):
    num = str(num)
    # 处理可能包含多个数字的情况，例如 "3.1, 3.2"
    parts = num.split(',')
    texts = []
    for part in parts:
        part = part.strip()  # 清除空白
        if part in conversion_dict:
            texts.extend(conversion_dict[part])
        else:
            texts.append('Unknown')
    return ', '.join(texts)

# 指定要转换的列索引
columns_to_convert = [11]  # Python索引为0开始，所以第5列是索引4，依此类推

# 应用转换函数到指定列
for col_index in columns_to_convert:
    df.iloc[:, col_index] = df.iloc[:, col_index].apply(convert_numbers_to_text)

# 保存修改后的 DataFrame 到新的 Excel 文件
df.to_excel('Label.xlsx', index=False)

In [ ]:
import pandas as pd

# 载入Excel文件
df = pd.read_excel('Answers_type2.xlsx')

# 定义转换关系字兄
conversion_dict = {
    'uv, ultravio': ['ultravio spectroscopy(uv)'],
    ('vis, visible'): ['Visible light spectroscopy(vis)'],
    ('nir, near-infrared'): ['near-infrared spectroscopy(nir)'],
    ('mir, mid-infrared'): ['mid-infrared spectroscopy(mir)'],
    ('ftir, Fourier transform infrared'): ['Fourier transform infrared spectroscopy(ftir)'],
    ('long-wave infrared, lwir, lir'): ['long-wave infrared spectroscopy(lir)'],
    ('raman, SERS'): ['Raman spectroscopy'],
    ('LIBS, Laser-induced breakdown, filament-induced breakdown spectroscopy'): ['Laser-induced breakdown spectroscopy(LIBS)'],
    ('terahertz, THz'): ['terahertz spectroscopy(THz)'],
    ('fluoresc, fluorescence, fluorometer'): ['fluorescence spectroscopy'],
    ('remote sensing, hyperspectral imaging'): ['hyperspectral imaging technology'],
    ('else',): ['else']
}

# 反向映射字典
reverse_dict = {}
for keys, value in conversion_dict.items():
    for key in keys:
        reverse_dict[key] = value[0]

# 函数，用于转换第四列的值
def convert_value(val):
    # 检查是否存在对应关系
    for key in reverse_dict:
        if key in val.lower():
            return reverse_dict[key]
    return val  # 如果没有找到对应关系，返回原始值

# 应用转换函数到第四列
df['Entity2:Spectrum'] = df['Entity2:Spectrum'].apply(convert_value)

# 保存修改后的数据到新的Excel文件中
df.to_excel('Answers_type2.xlsx', index=False)


TF-IDF检索

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 读取Excel文件
df1 = pd.read_excel('Answers_type2.xlsx')
df2 = pd.read_excel("Label.xlsx")

# 定义TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

# 准备语料库
corpus = [str(row[5]) + " " + str(row[11]) + " " + str(row[14]) + " " + str(row[6]).lower() for _, row in df2.iterrows()]

# 计算TF-IDF矩阵
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 定义余弦相似度检索函数
def cosine_retrieval(keywords, dataframe, top_n=1):
    # 将关键词转换为小写
    keywords = keywords.strip().lower()
    
    # 构建查询文本的TF-IDF向量
    query_vector = tfidf_vectorizer.transform([keywords])
    
    # 计算查询文本与语料库中每个文档的余弦相似度
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # 获取相似度最高的top_n行的索引
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 返回相关度最高的论文标题以及同一行第一列的值
    return dataframe.iloc[top_indices][dataframe.columns[0]]

# 遍历df1中每行的第三列值作为关键词进行检索
results = []
for index, row in df1.iterrows():
    keywords = str(row[2]) + " " + str(row[3])  # 第三、四列的索引
    relevant_rows = cosine_retrieval(keywords, df2)
    results.append(relevant_rows.values)

# 将结果转换为DataFrame并与df1合并
results_df = pd.DataFrame(results, columns=['NEW'])
df1 = pd.concat([df1, results_df], axis=1)

# 保存更新后的DataFrame到新的Excel文件
df1.to_excel('Answers_type2.xlsx', index=False)


In [ ]:
import os  
import pandas as pd  
from langchain.vectorstores import FAISS  
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  
from langchain.chains import RetrievalQA  
from langchain.chat_models import ChatOpenAI  
from langchain import PromptTemplate  
  
# 设置Chat-GPT API的密钥  
OPENAI_KEY = 'your key'  
  
# 加载Chat-GPT模型  
gpt_model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1, max_tokens=512, api_key=OPENAI_KEY)  
  
# 论文文件夹路径  
paper_folder = "F:\Code\Word_detecting\Vector_ku\Save_Vector_Store" 
  
# Excel文件路径  
excel_path = 'Answers_type2.xlsx'  # 替换为您的Excel文件路径  

# 加载Excel文件  
df = pd.read_excel(excel_path)  
  
# 用于保存答案的列表  
answers = []  
  
# Prompt模板  
prompt_template = """Based on the information of the faiss files of the related papers, briefly extract information from the text that is relevant to the question being asked.  
  
            known information:  
            {context}  
  
            Question:  
            {question}"""  
  
# 遍历Excel文件的每一行  
for index, row in df.iterrows():  
    paper_title = str(row.iloc[5])  # 假设第六列是论文标题  
    query_question = row.iloc[0]  # 假设第一列是查询问题  
      
    # 在文件夹中查找对应的.faiss文件
    vs_path = os.path.join(paper_folder, paper_title, 'Vector')
    faiss_path = [f for f in os.listdir(vs_path) if f.endswith('.faiss')]

    if faiss_path:  
        # 加载向量存储  
        vector_store = FAISS.load_local(vs_path,
                                    HuggingFaceEmbeddings(model_name="BAAI/bge-small-en"),
                                    index_name=paper_title,
                                    allow_dangerous_deserialization=True)
          
        # 创建RetrievalQA链  
        knowledge_chain = RetrievalQA.from_llm(llm=gpt_model,
                                           retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                           prompt=PromptTemplate(
                                               template=prompt_template,
                                               input_variables=["context", "question"]))
        result = knowledge_chain({"query": query_question})
        generated_question = result['result']
        print(paper_title,generated_question)
        answers.append(generated_question)  
    else:  
        # 如果找不到.faiss文件，添加一个占位符或错误信息  
        answers.append("No FAISS file found for this paper.")  
  
# 将答案添加到DataFrame中  
df['Answer'] = answers  
  
# 保存修改后的DataFrame回Excel文件  
df.to_excel(excel_path, index=False)

In [ ]:
import pandas as pd
import json

# 加载Excel文件
file_path = 'Answers_type2.xlsx'  # 请根据实际情况修改文件路径
sheet_name = 'Sheet1'  # 根据实际情况修改工作表名称
df = pd.read_excel(file_path, sheet_name=sheet_name)
# 准备JSON数据
json_data = []
for index, row in df.iterrows():
    question = row['Question']
    question_type = row['Question_type']  # 假设第二列是问题类型
    entity1 = row['Entity1:Object']  # 假设第三列是实体对象
    entity2=row['Entity2:Spectrum']
    Knowledge=row['Knowledge']
    answer = row['Answer']

    # 构造单个JSON对象
    json_object = {
        "Instruction":'Based on the information of Knowledge in this section, answer the question concisely and professionally. The content of the answer must be derived from the knowledge in this section, not randomly. The content in the answer must be the content in knowledge.',
        "Question": question,
        "Question_type": question_type,
        "Entity1:Object": entity1,
        "Entity2:Spectrum":entity2,
        "Knowledge":Knowledge,
        "Output": answer
    }
    json_data.append(json_object)

# 将数据输出为JSON格式
json_output = json.dumps(json_data, indent=4)
print(json_output)

# 如果需要，可以将JSON数据保存到文件中
with open('Answers_type2.json', 'w') as json_file:
    json_file.write(json_output)